In [ ]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15191 sha256=06eff5acb0e1c2a2f9117e4a3e994fb6effa37d6bc6c808049a3402d33e312b6
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [ ]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt

import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [ ]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [ ]:
# calling API 
data=YahooFinancials('BNB-USD').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
bnb_df = pd.DataFrame(data['BNB-USD']['prices'])

bnb_df =bnb_df .drop(['high','low','open','volume','date','adjclose'], axis=1)
bnb_df .rename(columns={'formatted_date':'date'},inplace=True)
bnb_df ['date']=pd.to_datetime(bnb_df['date'])


In [ ]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [ ]:
bnb_df

,close,date
0,20.646936,2019-11-07
1,19.690773,2019-11-08
2,19.735447,2019-11-09
3,20.481758,2019-11-10
4,20.133141,2019-11-11
...,...,...
1096,337.014557,2022-11-07
1097,327.690277,2022-11-08
1098,266.853119,2022-11-09
1099,304.080048,2022-11-10


In [ ]:
# Scale data for ML engine
scaler = MinMaxScaler()
bnb_df['scaled_close'] = scaler.fit_transform(np.expand_dims(bnb_df['close'].values, axis=1))

In [ ]:
def PrepareData(days):
    df = bnb_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [ ]:
PrepareData(3) # 3 days


(           close       date  scaled_close    future
 0      20.646936 2019-11-07      0.016901  0.016653
 1      19.690773 2019-11-08      0.015466  0.016130
 2      19.735447 2019-11-09      0.015533  0.017217
 3      20.481758 2019-11-10      0.016653  0.017806
 4      20.133141 2019-11-11      0.016130  0.017828
 ...          ...        ...           ...       ...
 1093  354.406372 2022-11-04      0.517817  0.491715
 1094  349.253723 2022-11-05      0.510084  0.477720
 1095  339.389374 2022-11-06      0.495279  0.386414
 1096  337.014557 2022-11-07      0.491715  0.442286
 1097  327.690277 2022-11-08      0.477720  0.425864
 
 [1098 rows x 4 columns], array([[0.46640465],
        [0.48076224],
        [0.5178168 ],
        [0.51008356],
        [0.49527887],
        [0.49171466],
        [0.4777205 ],
        [0.38641426],
        [0.44228557],
        [0.42586422]], dtype=float32), array([[[0.01690067455517342, Timestamp('2019-11-07 00:00:00')],
         [0.015465635937726156, Tim

In [ ]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [ ]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
137/137 [==============================] - 4s 9ms/step - loss: 0.0086
Epoch 2/80
137/137 [==============================] - 1s 8ms/step - loss: 0.0044
Epoch 3/80
137/137 [==============================] - 1s 8ms/step - loss: 0.0041
Epoch 4/80
137/137 [==============================] - 1s 8ms/step - loss: 0.0028
Epoch 5/80
137/137 [==============================] - 1s 10ms/step - loss: 0.0030
Epoch 6/80
137/137 [==============================] - 2s 14ms/step - loss: 0.0028
Epoch 7/80
137/137 [==============================] - 2s 13ms/step - loss: 0.0023
Epoch 8/80
137/137 [==============================] - 2s 13ms/step - loss: 0.0026
Epoch 9/80
137/137 [==============================] - 2s 13ms/step - loss: 0.0025
Epoch 10/80
137/137 [==============================] - 2s 15ms/step - loss: 0.0022
Epoch 11/80
137/137 [==============================] - 2s 13ms/step - loss: 0.0019
Epoch 12/80
137/137 [==============================] - 2s 13ms/step - loss: 0.0021
Epoch 13/80
137/1

In [ ]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (300.86$, 307.24$, 308.73$)


In [14]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
bnb_lstm=pd.DataFrame()
bnb_lstm['future days']=days
bnb_lstm['price']=predictions
bnb_lstm['ticker']='BNB'

bnb_lstm.to_csv('bnb_lstm.csv') 
files.download('bnb_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>